# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [5]:
df = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [6]:
# number of callbacks for black-sounding names
sum(df[df.race=='b'].call)

157.0

In [7]:
df.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [8]:
N = df.shape[0]
bN = sum(df.race == 'b')
wN = sum(df.race == 'w')
calls = sum(df.call)
b_calls = sum(df[df.race == 'b'].call)
w_calls = sum(df[df.race == 'w'].call)
callProp = calls / N
bcallProp = b_calls / bN
wcallProp = w_calls / wN

In [14]:
print('Proportion of all applicants called back: {0}'.format(round(callProp, 3)))
print('Proportion of white applicants called back: {0}'.format(round(wcallProp, 3)))
print('Proportion of black applicants called back: {0}'.format(round(bcallProp, 3)))

Proportion of all applicants called back: 0.08
Proportion of white applicants called back: 0.097
Proportion of black applicants called back: 0.064


## Q1

Here, a chi-square test is appropriate to determine if the proportion of callbacks for white-sounding and black-sounding names differs from the proportion of callbacks given to all applicants in general. Given that the variables of interest are categorical, they are not normally distributed, and CLT doesn't apply.

## Q2

null hypothesis: race doesn't affect callback rate

alternative hypothesis: callback rate for white and/or black sounding names differs from the overall callback rate

## Q3

In [18]:
# standard deviations of callbacks for the black and white samples
bc_std = df[df.race == 'b'].call.std()
wc_std = df[df.race == 'w'].call.std()
# standard error 
# (standard deviation of their respective sampling distributions)
bc_ste = bc_std / np.sqrt(bN)
wc_ste = wc_std / np.sqrt(wN)

In [24]:
raceDiff = wcallProp - bcallProp
steDiff = bc_ste + wc_ste
zstat = (raceDiff - 0) / (steDiff)
zstat
# Compute margin of error and confidence interval at 99% level
critical_z = st.norm.ppf(.99)
margin_error = critical_z * steDiff
print('Margin of error: {0}\n'.format(margin_error))
ci_low = round(raceDiff - margin_error, 2)
ci_high = round(raceDiff + margin_error, 2)
print('99% confidence interval: {0} - {1}'.format(ci_low,  ci_high))

Margin of error: 0.02550459339189762

99% confidence interval: 0.01 - 0.06


In [27]:
# create arrays of observed and expected frequencies for callbacks by race
obsF = np.array([b_calls, w_calls, bN-b_calls, wN-w_calls])
nocallProp = (1-callProp)
expF = np.array([bN*callProp, wN*callProp, bN*nocallProp, wN*nocallProp])

In [30]:
print('observed vs. expected frequencies')
obsF, expF

observed vs. expected frequencies


(array([  157.,   235.,  2278.,  2200.]),
 array([  196.,   196.,  2239.,  2239.]))

In [34]:
# compute chi-square test
chisq, p = st.chisquare(obsF, expF)
print('chi-square statistic, p-value')
chisq, p

chi-square statistic, p-value


(16.879050414270225, 0.00074839594410972541)

## Q4

The low p-value (*p*=0.0007) of the chi-sqaure test indicates that white vs black sounding names significantly impacts the likelihood that an applicant receives a callback from their resume submission. That is, the proportion of callbacks for each race significantly differs from the proportion of callbacks that applicants receive in general. 

## Q5

A word of caution, this analysis does not mean that race/name is the most important factor in receiving a callback for one's resume. This analysis intentionally only considered race to determine if there was an impact there at all. Other variables would have to be included in a logistic regression, for example, to ascertain the relative importance of race to other factors that may impact whether an applicant is called back.